In [2]:
import json

In [5]:
def get_exp(exp):
    root_dir = f'sm_local/outputs_{exp}'
    loggings = f"{root_dir}/loggings.json"
    with open(loggings, 'r') as f:
        loggings = json.load(f)

    extraction = loggings['extraction']
    avg_ext_valid = sum([_['valid_ratio'] * _['num_questions_used'] for _ in extraction]) / sum([_['num_questions_used'] for _ in extraction])
    avg_ext_num_persona = sum([_['total_num_personas_extracted'] for _ in extraction]) / len(extraction)
    avg_ext_time = sum([_['extraction_time'] for _ in extraction]) / len(extraction)
    # print('num_personas:', avg_ext_num_persona)
    # print('valid_ratio:', avg_ext_valid)
    # print('time:', avg_ext_time)
    # print()

    clustering = loggings['clustering']
    avg_cluster_time = sum([_['clustering_time'] for _ in clustering]) / len(clustering)

    # print('time:', avg_cluster_time)
    # print()

    summarizing = loggings['summarizing']
    avg_cluster_generated = sum([_['num_of_clusters'] for _ in summarizing]) / len(summarizing)
    avg_sum_persona_generated = sum([_['num_of_personas'] for _ in summarizing]) / len(summarizing)
    avg_sum_valid = sum([_['valid_ratio'] for _ in summarizing]) / len(summarizing)
    avg_sum_time = sum([_['summarizing_time'] for _ in summarizing]) / len(summarizing)
    # print('valid_ratio:', avg_sum_valid)
    # print('time:', avg_sum_time)
    # print()

    cleaning = loggings['cleaning']
    avg_clean_valid = sum([_['is_successful'] for _ in cleaning]) / len(cleaning)
    avg_clean_num_persona = sum([_['num_final_personas'] for _ in cleaning]) / len(cleaning)
    avg_clean_time = sum([_['cleaning_time'] for _ in cleaning]) / len(cleaning)
    # print('num_personas:', avg_clean_num_persona)
    # print('valid_ratio:', avg_clean_valid)
    # print('time:', avg_clean_time)

    res = [
        avg_ext_valid,
        avg_ext_num_persona,
        avg_ext_time,
        avg_cluster_generated,
        avg_cluster_time,
        avg_sum_persona_generated,
        avg_sum_valid,
        avg_sum_time,
        avg_clean_valid,
        avg_clean_num_persona,
        avg_clean_time
    ]

    return res

In [9]:
for llm in ['haiku', 'sonnet']:
    kmeans = 10
    survey = 'single'
    extraction_type = 'example'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_generated, avg_cluster_time, avg_sum_persona_generated, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_generated:.3f};{avg_cluster_time:.3f};{avg_sum_persona_generated:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")
    
    kmeans = 10
    survey = 'single'
    extraction_type = 'description'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_generated, avg_cluster_time, avg_sum_persona_generated, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_generated:.3f};{avg_cluster_time:.3f};{avg_sum_persona_generated:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")
   
    kmeans = 20
    survey = 'single'
    extraction_type = 'example'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_generated, avg_cluster_time, avg_sum_persona_generated, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_generated:.3f};{avg_cluster_time:.3f};{avg_sum_persona_generated:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")
   
    gmm = 10
    survey = 'single'
    extraction_type = 'example'
    exp = f'{llm}_gmm{gmm}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_generated, avg_cluster_time, avg_sum_persona_generated, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};gmm;{gmm};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_generated:.3f};{avg_cluster_time:.3f};{avg_sum_persona_generated:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")

    survey = 'single'
    extraction_type = 'example'
    exp = f'{llm}_dbscan_{survey}_{extraction_type}_test'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_generated, avg_cluster_time, avg_sum_persona_generated, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};dbscan;-1;{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_generated:.3f};{avg_cluster_time:.3f};{avg_sum_persona_generated:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")

    kmeans = 10
    survey = 'same_topic'
    extraction_type = 'example'
    exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_generated, avg_cluster_time, avg_sum_persona_generated, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_generated:.3f};{avg_cluster_time:.3f};{avg_sum_persona_generated:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")
    
    # kmeans = 10
    # survey = 'same_topic'
    # extraction_type = 'description'
    # exp = f'{llm}_kmeans{kmeans}_{survey}_{extraction_type}'
    # avg_ext_valid, avg_ext_num_persona, avg_ext_time, avg_cluster_time, avg_sum_valid, avg_sum_time, avg_clean_valid, avg_clean_num_persona, avg_clean_time = get_exp(exp)
    # print(f"{llm};kmeans;{kmeans};{survey};{extraction_type};{avg_ext_valid:.3f};{avg_ext_num_persona:.3f};{avg_ext_time:.3f};{avg_cluster_time:.3f};{avg_sum_valid:.3f};{avg_sum_time:.3f};{avg_clean_valid:.3f};{avg_clean_num_persona:.3f};{avg_clean_time:.3f}")
    

    

haiku;kmeans;10;single;example;1.000;737.600;383.647;10.000;81.111;41.467;0.960;29.767;0.867;33.000;35.057
haiku;kmeans;10;single;description;1.000;290.000;183.585;10.000;37.716;26.200;1.000;18.241;0.933;21.400;19.838
haiku;kmeans;20;single;example;1.000;740.000;385.673;20.000;81.937;47.667;1.000;34.058;0.667;22.733;51.028
haiku;gmm;10;single;example;1.000;734.400;382.350;10.000;363.321;39.400;0.960;29.717;0.800;27.133;38.241
haiku;dbscan;-1;single;example;1.000;740.000;395.318;5.133;81.221;5.467;0.933;5.103;0.933;5.333;4.848
haiku;kmeans;10;same_topic;example;1.000;2822.000;1487.892;10.000;310.600;46.000;0.800;48.685;0.667;22.000;50.041
sonnet;kmeans;10;single;example;1.000;682.600;1058.366;10.000;66.282;20.533;1.000;55.342;0.867;17.133;40.706
sonnet;kmeans;10;single;description;0.998;247.400;534.450;10.000;31.378;13.533;0.967;40.696;0.933;11.933;27.270
sonnet;kmeans;20;single;example;1.000;682.000;1060.253;20.000;66.026;26.000;1.000;77.472;0.867;21.800;52.923
sonnet;gmm;10;single;exa

In [12]:
import pandas as pd

# get file path under sm_local
model = 'haiku'
for kmeans in ['10']:
    for level in ['high','mid','low']:
        file_name = f"sm_local/outputs_{model}_kmeans{kmeans}_single_example/cleaned/cleaned_{level}_level_personas_American_Trends_Panel_W26.json"
        df = pd.read_json(file_name)
        df.to_csv(f'{model}_kmeans_{kmeans}_single_example_W26_{level}.csv')

In [17]:
# list all folders in 'pump/sm_local'

import os

folders = [_ for _ in os.listdir('sm_local') if _.startswith('outputs') and 'meanshift' not in _]
folders

['outputs_haiku_kmeans20_single_example',
 'outputs_sonnet_kmeans10_single_example',
 'outputs_haiku_dbscan_single_example_test',
 'outputs_sonnet_gmm10_single_example',
 'outputs_sonnet_kmeans10_same_topic_description',
 'outputs_sonnet_kmeans20_single_example',
 'outputs_sonnet_kmeans10_single_description',
 'outputs_haiku_gmm10_single_example',
 'outputs_haiku_kmeans10_same_topic_description',
 'outputs_haiku_kmeans10_same_topic_example',
 'outputs_haiku_kmeans10_single_example',
 'outputs_haiku_kmeans10_single_description',
 'outputs_sonnet_kmeans10_same_topic_example',
 'outputs_sonnet_dbscan_single_example_test']

In [27]:
for config in folders:
    cleaned_dir = f"sm_local/{config}/cleaned"
    print(cleaned_dir)

    for filename in os.listdir(cleaned_dir):
        exp = filename[:-5]
        os.makedirs(f"sm_local/results/{config}/", exist_ok=True)
        df = pd.read_json(f"sm_local/{config}/cleaned/{exp}.json")
        df.to_csv(f"sm_local/results/{config}/{exp}.csv")
        df.to_json(f"sm_local/results/{config}/{exp}.json", indent=4, orient="records")

sm_local/outputs_haiku_kmeans20_single_example/cleaned
sm_local/outputs_sonnet_kmeans10_single_example/cleaned
sm_local/outputs_haiku_dbscan_single_example_test/cleaned
sm_local/outputs_sonnet_gmm10_single_example/cleaned
sm_local/outputs_sonnet_kmeans10_same_topic_description/cleaned
sm_local/outputs_sonnet_kmeans20_single_example/cleaned
sm_local/outputs_sonnet_kmeans10_single_description/cleaned
sm_local/outputs_haiku_gmm10_single_example/cleaned
sm_local/outputs_haiku_kmeans10_same_topic_description/cleaned
sm_local/outputs_haiku_kmeans10_same_topic_example/cleaned
sm_local/outputs_haiku_kmeans10_single_example/cleaned
sm_local/outputs_haiku_kmeans10_single_description/cleaned
sm_local/outputs_sonnet_kmeans10_same_topic_example/cleaned
sm_local/outputs_sonnet_dbscan_single_example_test/cleaned
